In [30]:
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.models import Model

from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras import applications

import os
import numpy as np

In [15]:
np.random.seed(0)
tf.random.set_seed(0)

In [16]:
pwd

'/home/ec2-user'

In [28]:
# 변수 정의
train_dir = '/home/ec2-user/images/train'

input_size = 224
batch_size = 16
epochs = 20


In [19]:
# 모델 불러오기
vgg_model = VGG16(input_shape=(input_size, input_size,3), weights='imagenet', include_top=False)

In [20]:
vgg_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [21]:
vgg_model.trainable=True
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()

model = tf.keras.Sequential([
    vgg_model,
    global_average_layer,
    layers.Dense(19, activation='softmax'),

])

In [22]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 7, 7, 512)         14714688  
_________________________________________________________________
global_average_pooling2d_1 ( (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 19)                9747      
Total params: 14,724,435
Trainable params: 14,724,435
Non-trainable params: 0
_________________________________________________________________


In [23]:
model.compile(loss='categorical_crossentropy',
                     optimizer='adam',
                     metrics=['accuracy'])

In [24]:
pwd

'/home/ec2-user'

In [25]:

image_generator = ImageDataGenerator(validation_split=0.2, rescale=1./255)

# 데이터 구조 생성
train_data_gen = image_generator.flow_from_directory(batch_size=batch_size,
                                                           directory=train_dir,
                                                           subset='training',
                                                           shuffle=True,
                                                           target_size=(input_size, input_size),
                                                           class_mode='categorical')

valid_data_gen = image_generator.flow_from_directory(batch_size=batch_size,
                                                         directory=train_dir,
                                                         subset='validation',
                                                         target_size=(input_size, input_size),
                                                         class_mode='categorical')

Found 839 images belonging to 19 classes.
Found 201 images belonging to 19 classes.


In [27]:
# weights 경로 설정 및 callback 설정

checkpoint_path = "/home/ec2-user/weights/cp.ckpt"
checkpoint_dir = os.path.join(checkpoint_path)
cp_callback=tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                               save_weights_only=True,
                                               verbose=1)

In [29]:
# 모델 학습
history = model.fit(train_data_gen, epochs=epochs, validation_data=valid_data_gen, callbacks=[cp_callback])

Epoch 1/20
53/53 [==============================] - ETA: 0s - loss: 3.5336 - accuracy: 0.0620
Epoch 00001: saving model to /home/ec2-user/weights/cp.ckpt
53/53 [==============================] - 42s 797ms/step - loss: 3.5336 - accuracy: 0.0620 - val_loss: 2.9253 - val_accuracy: 0.0498
Epoch 2/20
53/53 [==============================] - ETA: 0s - loss: 2.9324 - accuracy: 0.0870
Epoch 00002: saving model to /home/ec2-user/weights/cp.ckpt
53/53 [==============================] - 27s 506ms/step - loss: 2.9324 - accuracy: 0.0870 - val_loss: 2.9286 - val_accuracy: 0.0995
Epoch 3/20
53/53 [==============================] - ETA: 0s - loss: 2.9232 - accuracy: 0.0954
Epoch 00003: saving model to /home/ec2-user/weights/cp.ckpt
53/53 [==============================] - 23s 426ms/step - loss: 2.9232 - accuracy: 0.0954 - val_loss: 2.8995 - val_accuracy: 0.0995
Epoch 4/20
53/53 [==============================] - ETA: 0s - loss: 2.9081 - accuracy: 0.0989
Epoch 00004: saving model to /home/ec2-user/weig

In [31]:
# test 데이터 셋

img = load_img('/home/ec2-user/jaesuk.jpg', target_size=(input_size, input_size))
img_np = img_to_array(img)
img_np = img_np[np.newaxis, ...]


In [32]:
# Keras VGG16 모델에서 했던 전처리 방식을 적용
img_tensor = applications.vgg16.preprocess_input(img_np) 
img_tensor.shape

(1, 224, 224, 3)

In [33]:
pred = model.predict(img_tensor)

In [34]:
pred

array([[2.63840934e-06, 4.49768326e-04, 8.01756699e-03, 6.19033992e-04,
        9.41848218e-01, 4.46279086e-02, 2.66253861e-04, 1.13907117e-06,
        5.08122355e-07, 2.70956225e-05, 1.38693414e-11, 3.16540129e-04,
        4.13368326e-08, 5.82713801e-06, 1.40545972e-05, 1.82258417e-10,
        1.57485530e-07, 3.80281475e-03, 2.73950718e-07]], dtype=float32)

In [35]:
pred_label = np.argmax(pred, axis=1)
pred_label

array([4])

In [37]:
pred_label1 = np.where(pred > 0.5, 1, 0)
pred_label1

array([[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [38]:
print( pred_label)
valid_data_gen.class_indices,

[4]


({'JungHae-in': 0,
  'Jungwoosung': 1,
  'Lee_Jung-jae': 2,
  'Seojin_Lee': 3,
  'Song_gang_ho': 4,
  'Song_joong_Gi': 5,
  'Yoo_Jae_Suk': 6,
  'baedoona': 7,
  'hansohee': 8,
  'kimtaehee': 9,
  'kimyeonkyoung': 10,
  'kimyuna': 11,
  'leejiah': 12,
  'leesungkyoung': 13,
  'parkseojoon': 14,
  'parksodam': 15,
  'songhyekyo': 16,
  'won_bin': 17,
  'younyuhjung': 18},)

In [110]:
new_model = VGG16(input_shape=(224,224,3), weights='imagenet', include_top=False)

In [111]:
vgg_model.trainable=True
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()

new_model = tf.keras.Sequential([
    vgg_model,
    global_average_layer,
    layers.Dense(19, activation='softmax'),

])

In [112]:
new_model.compile(loss='categorical_crossentropy',
                     optimizer='adam',
                     metrics=['accuracy'])

In [113]:
weights_path = "/content/drive/MyDrive/crime_project/weights"
weights_file_path = os.path.join(weights_path, "cp.ckpt")


In [114]:
new_model.load_weights(weights_file_path)

In [115]:
new_pred = new_model.predict(img_tensor)
new_pred

array([[1.03670184e-10, 4.53466855e-05, 3.76508832e-01, 2.12066844e-02,
        1.64957037e-05, 7.38900229e-02, 5.70175564e-03, 6.57197106e-08,
        2.01494157e-01, 8.82752836e-02, 2.94074964e-09, 3.10313189e-05,
        2.22430909e-08, 1.04333833e-06, 1.38426376e-05, 4.47008356e-14,
        2.32600987e-01, 2.14443673e-04, 1.02289271e-12]], dtype=float32)

In [117]:
new_pred_label = np.argmax(pred, axis=1)
new_pred_label

array([2])

In [119]:
new_pred_labl1 = np.where(pred > 0.5, 1, 0)
new_pred_labl1

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [78]:
valid_data_gen.class_indices


{'JungHae-in': 0,
 'Jungwoosung': 1,
 'Lee_Jung-jae': 2,
 'Seojin_Lee': 3,
 'Song_gang_ho': 4,
 'Song_joong_Gi': 5,
 'Yoo_Jae_Suk': 6,
 'baedoona': 7,
 'hansohee': 8,
 'kimtaehee': 9,
 'kimyeonkyoung': 10,
 'kimyuna': 11,
 'leejiah': 12,
 'leesungkyoung': 13,
 'parkseojoon': 14,
 'parksodam': 15,
 'songhyekyo': 16,
 'won_bin': 17,
 'younyuhjung': 18}

In [120]:
new_pred == pred

array([[ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True]])